#### Svrha bilježnice: kreiranje skupa {članak : vektor}

In [2]:
import pandas as pd
import jsonlines
import gc 
#import gensim 
#from gensim.models.wrappers import fasttext
import fasttext.util
import re
import spacy
import nltk
from nltk.corpus import stopwords
from collections import Counter
import numpy as np
from string import punctuation
import sys
import re 
import random
import json
import time
import tqdm 
from tqdm import tqdm

In [3]:
import fasttext.util
#fasttext.util.download_model('en', if_exists='ignore')  # English
ft = fasttext.load_model('../../datasets/cc.en.300.bin')

In [5]:
#!python -m spacy download en_core_web_lg

In [6]:
#fasttext.util.download_model('en', if_exists='ignore')  # ako nemamo lokalno
#ft = fasttext.load_model('cc.en.300.bin')
nlp = spacy.load("en_core_web_lg",disable=['parser', 'tagger', 'ner'])

In [7]:
#nltk.download('stopwords')

In [8]:
df = pd.read_parquet('../../datasets/dblpv13.df.parquet')


In [9]:
df.head(2)

,id,categories,references,title,keywords,url,abstract
0,53e99784b7602d9701f3e133,"[Agronomy, Moisture, Hydrology, Environmental ...",None,The relationship between canopy parameters and...,"[canopy parameters, canopy spectrum, different...",[http://dx.doi.org/10.1109/IGARSS.2011.6049503],Drought is the first place in all the natural ...
1,53e99784b7602d9701f3e15d,"[Delay calculation, Timing failure, Monte Carl...","[53e9a8a9b7602d97031f6bb9, 599c7b6b601a182cd27...",Timing yield estimation using statistical stat...,"[sequential circuits, statistical distribution...","[http://dx.doi.org/10.1109/ISCAS.2005.1465124,...",As process variations become a significant pro...


In [4]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\@\-|,;]|(\\n)')
COPYRIGHT_RE = re.compile('©[^\.]*.')
BAD_SYMBOLS_RE = re.compile('[^a-z \'\-]')
STOPWORDS = set(stopwords.words('english'))
SUCC_SPACES_RE = re.compile(' +') 

In [5]:
def text_prepare(text, tokenizer):
  
    text = text.lower() # lowercase text
    text= text.replace('all rights reserved.',' ')
    text= COPYRIGHT_RE.sub(' ',text)
    
    text = REPLACE_BY_SPACE_RE.sub(' ',text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('',text)# delete symbols which are in BAD_SYMBOLS_RE from text
    text = SUCC_SPACES_RE.sub(' ',text)
    text = ' '.join([token.text for token in tokenizer(text) if token.text not in STOPWORDS and len(token.text)>2])# delete stopwords from text
    return text #modified initial string
    

In [10]:
npd=df.to_numpy() 

In [23]:
pbar = tq.tqdm(total=npd.shape[0])
preprocessed=[]

for row in npd:
    txt=""
    if row[1] is not None:
        txt=' '.join(row[1])#kategorije
    if row[3] is not None:
        txt=' '.join([txt,row[3]])#naslov
    if row[4] is not None:
        txt=' '.join([txt,' '.join(row[4])])#ključne riječi
    txt+=" "+row[6] #sažetak

    preprocessed.append(text_prepare(txt,nlp.tokenizer))

    pbar.update(1)

pbar.close()

100%|███████████████████████████████| 3851441/3851441 [12:56<00:00, 4959.90it/s]


In [24]:
df = df[['id']] 

In [26]:
df['preprocessed']=preprocessed

In [14]:
df.head()

,id,preprocessed
0,53e99784b7602d9701f3e133,agronomy moisture hydrology environmental scie...
1,53e99784b7602d9701f3e15d,delay calculation timing failure monte carlo m...
2,53e99784b7602d9701f3e161,global high technology daily short distance fl...
3,53e99784b7602d9701f3e4f4,btextures visual source material minute sound ...
4,53e99784b7602d9701f3eaf2,monkeysii constructing system cope dynamically...


In [28]:
preprocessed[0]

'agronomy moisture hydrology environmental science dry weight water content stomatal conductance transpiration irrigation soil water canopy relationship canopy parameters spectrum winter wheat different irrigations hebei province canopy parameters canopy spectrum different soil water content control winter wheat irrigation hydrology radiometry moisture indexes vegetation indexation dry weight soil moisture water content indexing terms spectrum natural disaster drought first place natural disasters world especially serious north china plain paper different soil water content control levels winter wheat growth stages performed gucheng ecological meteorological integrated observation experiment station cams china canopy parameters including growth conditions dry weight physiological parameters hyperspectral reflectance measured erecting stage milk stage winter wheat relationship canopy parameters soil relative moisture canopy water content water indices winter wheat established results sh

In [43]:
!pwd

/home/dora/Desktop/recommender/helper_notebooks/zero_step_notebooks


In [30]:
#df.to_parquet('preprocessed.parquet', engine='pyarrow', compression='gzip')
#pd.DataFrame(zip(ids,preprocessed),columns=['id', 'preprocessed']).to_json('../../datasets/preprocessed.json") #spremamo

In [6]:
df = pd.read_parquet('../../datasets/preprocessed.parquet', engine='pyarrow')

In [25]:
preprocessed=npd[:,1]

In [23]:
def get_me_some_vectors(text):  
    
    words = text.split()   #rastavlja text na rijeci
    vectors = 0
    numOfExceptableWords = 0  #ukupni broj rijeci za koje cemo izgenerirati vektore
    for w in words:
      try:                    #ako neka rijec ne postoji u vokabularu baca se iznimka
        vec = ft.get_word_vector(w)
        vectors += vec
        numOfExceptableWords +=1
      except:
        continue
  
    return vectors/numOfExceptableWords, numOfExceptableWords
    


In [26]:
#svaki predprocesirani clanak postaje fasttext vektor dimenzije 300 
pbar = tq.tqdm(total=npd.shape[0])

ftVectors=[]
now_all=[]
i=0
for txt in preprocessed:
    vct,now=get_me_some_vectors(txt)
    ftVectors.append(vct)
    now_all.append(now)
    #ftVectors.append(ft.get_sentence_vector(txt))
    pbar.update(1)

pbar.close()

  0%| | 0/3851441 [00:1
100%|█| 3851441/3851441


In [27]:
df=df.drop(['preprocessed'],axis=1)
df['vector']=ftVectors 
df['num_of_words']=now_all 


In [28]:
df.head()

,id,vector,num_of_words
0,53e99784b7602d9701f3e133,"[-0.44870365, 3.0942643, -0.19170868, 10.43203...",197
1,53e99784b7602d9701f3e15d,"[-1.6448469, 2.4118207, 3.2933245, 12.960641, ...",224
2,53e99784b7602d9701f3e161,"[-0.21152683, 0.7793277, -0.52211463, 1.900005...",32
3,53e99784b7602d9701f3e4f4,"[-0.45753658, -0.30740556, 3.26288, 4.5930157,...",76
4,53e99784b7602d9701f3eaf2,"[-0.8303563, -0.54809356, 0.38332126, 4.590071...",76


In [29]:
df.to_parquet('../../datasets/dblpv13_fasttext.parquet', engine='pyarrow', compression='gzip')